In [1]:
from pathlib import Path
from typing import Literal
from warnings import warn

import scipy.stats as stats
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

from scipy.ndimage import label

from astropy.coordinates import SkyCoord
from astropy import units as u
from astropy.wcs import WCS
import astropy.visualization as vis

from lsst.geom import SpherePoint, Point2I
import lsst.geom as geom
from lsst.daf.butler import Butler
from lsst.rsp import get_tap_service
from lsst.afw.math import Warper, WarperConfig
from lsst.source.injection import generate_injection_catalog
from lsst.source.injection import VisitInjectConfig, VisitInjectTask
from lsst.ip.diffim.subtractImages import AlardLuptonSubtractTask, AlardLuptonSubtractConfig
from lsst.ip.diffim.detectAndMeasure import DetectAndMeasureTask, DetectAndMeasureConfig

In [2]:
service = get_tap_service("tap")

# Source detection over multiple simulated afterglows

We put together the code for anomaly detection we developed previously and execute it over our simulated afterglow events.

## Detect, table, annotate pipeline

In [3]:
def detect(science, template, difference, threshold: float | None = None):
    config = DetectAndMeasureConfig()
    if threshold is not None:
        config.detection.thresholdValue = threshold
    anomalies = DetectAndMeasureTask(config=config).run(science, template, difference)
    _df = anomalies.getDict()["diaSources"].asAstropy().to_pandas()
    _d = {
        "coord_ra": np.rad2deg(_df["coord_ra"]),
        "coord_dec": np.rad2deg(_df["coord_dec"]),
        "PsfFlux_base": _df["base_PsfFlux_instFlux"],
        "PsfFlux_baseErr": _df["base_PsfFlux_instFluxErr"],
        "PsfFlux_diffim": _df["ip_diffim_forced_PsfFlux_instFlux"],
        "PsfFlux_diffimErr": _df["ip_diffim_forced_PsfFlux_instFluxErr"],
        "snr_diffim": _df["ip_diffim_forced_PsfFlux_instFlux"] / _df["ip_diffim_forced_PsfFlux_instFluxErr"],
        "snr_base": _df["base_PsfFlux_instFlux"] / _df["base_PsfFlux_instFluxErr"],
        "mag_base": [
            science.getPhotoCalib().instFluxToMagnitude(f) 
            for f in _df["base_PsfFlux_instFlux"]
        ],
        "mag_base_errBot": [
             -2.5 * np.log10(1 + delta_f / f)
            for f, delta_f in zip(_df["base_PsfFlux_instFlux"], _df["base_PsfFlux_instFluxErr"],)
        ],
        "mag_base_errTop": [
             -2.5 * np.log10(1 - delta_f / f)
            for f, delta_f in zip(_df["base_PsfFlux_instFlux"], _df["base_PsfFlux_instFluxErr"],)
        ],
        "mag_diffim": [
            science.getPhotoCalib().instFluxToMagnitude(f) 
            for f in _df["ip_diffim_forced_PsfFlux_instFlux"]
        ],
        "mag_diffim_errBot": [
             -2.5 * np.log10(1 + delta_f / f)
            for f, delta_f in zip(_df["ip_diffim_forced_PsfFlux_instFlux"], _df["ip_diffim_forced_PsfFlux_instFluxErr"],)
        ],
        "mag_diffim_errTop": [
             -2.5 * np.log10(1 - delta_f / f)
            for f, delta_f in zip(_df["ip_diffim_forced_PsfFlux_instFlux"], _df["ip_diffim_forced_PsfFlux_instFluxErr"],)
        ],
        "is_negative": _df["is_negative"],
    }
    return pd.DataFrame(_d).sort_values("snr_base", ascending=False).reset_index(drop=True)


ANNOTATION_PREFIX = "ann"

def annotate_donuts(table_df: pd.DataFrame) -> pd.DataFrame:
    mask = table_df["PsfFlux_base"] < 0
    return pd.concat([table_df, pd.DataFrame({f"{ANNOTATION_PREFIX}_donut": mask})], axis=1)


def annotate_mask(table_df: pd.DataFrame, difference) -> pd.DataFrame:
    keywords: list[list[str]] = [get_mask_labels_coord(difference, ra, dec) for ra, dec in table_df[["coord_ra", "coord_dec"]].values]
    return pd.concat(
        (table_df,
        pd.DataFrame({
                f"{ANNOTATION_PREFIX}_mask_{k.lower()}": [k in ks for ks in keywords ]
                for k in set([k for ks in keywords for k in ks])
        })),
        axis=1,
    ).reset_index(drop=True)


def annotate_star_closeness(
    table_df: pd.DataFrame,
    star_catalog: pd.DataFrame,
    tolerance_arcsec: float = 0.2, # pixel size
) -> pd.DataFrame:
    source_coords = table_df[["coord_ra", "coord_dec"]].values
    ref_coords = star_catalog[["coord_ra", "coord_dec"]].values
    tolerance_deg = np.deg2rad(tolerance_arcsec / 3600.0)
    s_ = np.deg2rad(source_coords[:, np.newaxis, :])
    r_ = np.deg2rad(ref_coords[np.newaxis, :, :])
    distances = np.acos(np.sin(s_[:, :, 1]) * np.sin(r_[:, :, 1]) + np.cos(s_[:, :, 1]) * np.cos(r_[:, :, 1]) * np.cos(s_[:, :, 0] - r_[:, :, 0]))
    mask = np.any(distances  < tolerance_deg, axis=1)
    return pd.concat([table_df, pd.DataFrame({f"{ANNOTATION_PREFIX}_closestar": mask})], axis=1)


# TODO: improve this weak implementation
def annotate_ext_overlap(
    table_df: pd.DataFrame,
    science,
    galaxies_catalog: pd.DataFrame,
)-> pd.DataFrame:
    mask_detected = get_mask(science, "DETECTED").astype(int)
    labelled, nlabels = label(mask_detected)
    out = np.zeros_like(labelled).astype(bool)
    # TODO: improve this implementation vectorizing it
    for i, source in galaxies_catalog.iterrows():
        ra, dec = source[["coord_ra", "coord_dec"]].values
        if not science.containsSkyCoords(ra * u.deg, dec * u.deg):
            continue
        pixel = coord_to_pix(science, ra, dec)
        value = labelled[*pixel]
        if value == 0:
            continue
        out |= (labelled == value)
    mask = table_df[["coord_ra", "coord_dec"]].apply(
        lambda x: out[*coord_to_pix(science, x.coord_ra, x.coord_dec)], 
        axis=1
    )
    return pd.concat([table_df, pd.DataFrame({f"{ANNOTATION_PREFIX}_extoverlap": mask})], axis=1)
    

### Mask Tools

In [4]:
def get_mask(image, mask_names: str | list[str]) -> np.ndarray:
    mask = image.getMask()
    if isinstance(mask_names, str):
        mask_names = [mask_names]

    mask_array = mask.getArray()
    out = np.zeros_like(mask_array)
    for mask_name in mask_names:
        target_bit = mask.getMaskPlane(mask_name)
        out |= (mask_array & (2 ** target_bit)) != 0
    return out

def coord_to_pix(image, ra, dec) -> tuple[int, int]:
    wcs = WCS(image.getWcs().getFitsMetadata())
    coord = SkyCoord(
        ra = ra * u.deg,
        dec = dec * u.deg
    )
    i, j = wcs.world_to_array_index(coord)
    # apparently can return pixel at boundary
    return min(i, image.getHeight() - 1), min(j, image.getWidth() - 1), 

def get_mask_value_coord(image, ra, dec) -> int:
    return image.getMask()[*coord_to_pix(image, ra, dec)[::-1]]

def get_mask_labels_coord(image, ra, dec) -> list[str]:
    return [
        keyword 
        for keyword, bit in image.getMask().getMaskPlaneDict().items() 
        if get_mask_value_coord(image, ra, dec) & (2 ** bit) != 0
    ]

### Catalog helpers

In [5]:
def sregion_to_vertices(sregion: str, closed=False):
    """Convert the s_region from the ObsCore table into two
    arrays containing the x and y vertices, in order to plot
    boxes using matplotlib.

    from notebook DPO02_02c.
    """
    temp = sregion.split(' ')
    xvertices = []
    yvertices = []
    ix = 2
    iy = 3
    for c in range(4):
        xvertices.append(float(temp[ix]))
        yvertices.append(float(temp[iy]))
        ix += 2
        iy += 2
    if closed:
        xvertices.append(xvertices[0])
        yvertices.append(yvertices[0])
    return list(zip(xvertices, yvertices))


def polygon_string(sregion: str):
    """
    Usage:
        * Plug in queries like:
        
            ```
            (.. more query ..)
            WHERE CONTAINS(POINT('ICRS', obj.coord_ra, obj.coord_dec), {polygon_string(s_region)}) = 1
            (.. more query ..)                                          ^^^^^^^^^^^^^^^^^^^^^^^^
            ```
    """
    return f"""POLYGON('ICRS', {
    ', '.join(map(
        lambda x: f'{x[0]:.6f}, {x[1]:.6f}', 
        sregion_to_vertices(sregion, False),
    ))})"""


def calexp_ccdvisitid(calexp) -> str:
    detector = calexp.detector.getId()
    id_ = calexp.visitInfo.id
    ccdvisitid = f"{id_}{detector:03d}"
    return ccdvisitid


def sources_in(service, calexp, extendedness: bool | None = None) -> pd.DataFrame:
    band = calexp.getFilter().bandLabel
    ccdvisitid = calexp_ccdvisitid(calexp)

    query_sregion = (
        f"""
        SELECT s_region 
        FROM dp02_dc2_catalogs.ObsCore 
        WHERE lsst_ccdvisitid = {ccdvisitid} 
        AND dataproduct_subtype = 'lsst.calexp'
        """
    )
    s_region = (_t := service.search(query_sregion).to_table())[0][0]
    assert len(_t) == 1
    and_extendedness = f"AND obj.{band}_extendedness = {int(extendedness)}" if extendedness is not None else ""
    query_objects = (
        f"""
        SELECT obj.coord_ra AS coord_ra,
        obj.coord_dec AS coord_dec,
        obj.{band}_extendedness AS {band}_extendedness,
        scisql_nanojanskyToAbMag(obj.{band}_cModelFlux) AS cModelMag_{band}, 
        obj.{band}_cModelFlux AS {band}_cModelFlux,
        obj.footprintArea AS footprintArea 
        FROM dp02_dc2_catalogs.Object AS obj 
        WHERE CONTAINS(POINT('ICRS', obj.coord_ra, obj.coord_dec), {polygon_string(s_region)}) = 1
        AND obj.detect_isPrimary = 1 
        {and_extendedness}
        """
    )
    return service.search(query_objects).to_table().to_pandas()

In [6]:
def detection_pipeline(science, template, difference, threshold: float | None = None, footprint_area: int | None = None) -> pd.DataFrame:
    print("Starting source detection.")
    anomalies = detect(science, template, difference)
    sources = sources_in(service, science)
    pointlike_sources = sources[sources["r_extendedness"] == 0]
    results = annotate_star_closeness(
        annotate_mask(
            annotate_donuts(anomalies), 
            difference,
        ),
        pointlike_sources,
    )
    if footprint_area is None:
        return results
    extended_sources = sources[sources["footprintArea"] >= footprint_area]
    return annotate_ext_overlap(results, template, extended_sources)


# Testing

In [7]:
from doppelganger.doppelganger.rubin.inject import sfis_pipeline

In [8]:
def test_detection_pipeline():
    ra = 56.90063
    dec = -33.94851
    mag = 17.
    band = "r"
    footprint_area = 4000
    
    results = sfis_pipeline(service, ra, dec, mag, band)
    return detection_pipeline(**results, footprint_area = 4000)

annotated_anomalies = test_detection_pipeline()

Retrieving visit table, choosing one visit at random.
Starting source injection.


Starting DIA.


Starting source detection.


/tmp/ipykernel_24377/3527383882.py:21: RuntimeWarning: invalid value encountered in log10
  -2.5 * np.log10(1 + delta_f / f)
/tmp/ipykernel_24377/3527383882.py:25: RuntimeWarning: invalid value encountered in log10
  -2.5 * np.log10(1 - delta_f / f)
/tmp/ipykernel_24377/3527383882.py:74: RuntimeWarning: invalid value encountered in arccos
  distances = np.acos(np.sin(s_[:, :, 1]) * np.sin(r_[:, :, 1]) + np.cos(s_[:, :, 1]) * np.cos(r_[:, :, 1]) * np.cos(s_[:, :, 0] - r_[:, :, 0]))


In [9]:
annotated_anomalies

,coord_ra,coord_dec,PsfFlux_base,PsfFlux_baseErr,PsfFlux_diffim,PsfFlux_diffimErr,snr_diffim,snr_base,mag_base,mag_base_errBot,...,ann_mask_injected_core,ann_mask_intrp,ann_mask_detected_negative,ann_mask_crosstalk,ann_mask_sat_template,ann_mask_detected,ann_mask_sat,ann_mask_injected,ann_closestar,ann_extoverlap
0,56.900630,-33.948510,1.175494e+06,1562.572519,1.181909e+06,1571.357612,752.157970,752.281050,17.002640,-0.001442,...,True,False,False,False,False,True,False,True,False,True
1,56.862229,-33.952394,2.438484e+03,240.047436,5.458884e+03,237.714051,22.964079,10.158343,23.710391,-0.101942,...,False,False,False,False,False,True,False,False,False,False
2,57.103582,-33.850806,1.798593e+03,230.489362,1.729306e+03,226.629108,7.630557,7.803366,24.040859,-0.130917,...,False,True,False,False,False,True,True,False,False,False
3,57.005422,-33.985053,6.732509e+03,882.228750,3.577158e+05,871.437776,410.489228,7.631251,22.607749,-0.133695,...,False,False,False,False,False,True,False,False,True,True
4,57.115374,-33.833888,1.453799e+03,228.494298,1.448863e+03,225.298555,6.430857,6.362516,24.271931,-0.158493,...,False,True,False,False,False,True,True,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
69,57.003522,-33.812200,-1.579815e+04,1403.500634,8.960107e+05,1381.295239,648.674319,-11.256248,NaN,0.101013,...,False,False,True,False,False,True,False,False,True,True
70,56.900939,-33.942134,-1.412594e+05,2013.777336,1.933219e+06,1984.202423,974.305164,-70.146486,NaN,0.015590,...,False,True,True,False,True,False,True,False,False,True
71,56.934929,-33.831691,-2.933909e+05,2011.829331,1.917899e+06,1948.543907,984.273057,-145.832886,NaN,0.007471,...,False,True,True,False,True,False,True,False,False,True
72,57.100552,-33.824742,-3.232499e+05,1966.533268,1.870760e+06,1912.506801,978.171589,-164.375506,NaN,0.006625,...,False,True,True,False,True,False,True,False,False,True


# Operations

In [8]:
np.random.seed(0)

In [9]:
events_df = pd.read_csv("data/06_sourceCatalogSim/afterglow_host.csv")
for i, (ra, dec, mag) in events_df[["ra", "dec", "m"]].iterrows():
    print(f"START SIMULATION of source with index {i:02d}, ra = {ra:.4f}, dec = {dec:.4f}, m = {mag:.4f}")
    results = sfis_pipeline(service, ra, dec, mag, "r")
    annotated_anomalies = detection_pipeline(**results, footprint_area = 4000)
    ccdvisitid = calexp_ccdvisitid(results['science'])
    rastr, decstr = map(lambda x: f"{x:.3f}".replace(".", "d").replace("-", "m"), [ra, dec])
    magstr = f"{mag:.1f}".replace(".", "d")
    idstring = f"i{i:02d}_ccdvisitid{ccdvisitid}_ra{rastr}_dec{decstr}_mag{magstr}"
    
    for k in ["science", "template", "difference"]:
        results[k].writeFits(f"data/08_anomalies/anomalies_{k}_{idstring}.fits")
    annotated_anomalies.to_csv(f"data/08_anomalies/anomalies_{idstring}.csv")
    print("ENDS SIMULATION.\n\n")

START SIMULATION of source with index 00, ra = 68.0377, dec = -40.5540, m = 23.8358
Retrieving visit table, choosing one visit at random.
Starting source injection.


Starting DIA.


Starting source detection.


/tmp/ipykernel_33451/3527383882.py:21: RuntimeWarning: invalid value encountered in log10
  -2.5 * np.log10(1 + delta_f / f)
/tmp/ipykernel_33451/3527383882.py:25: RuntimeWarning: invalid value encountered in log10
  -2.5 * np.log10(1 - delta_f / f)


ENDS SIMULATION.


START SIMULATION of source with index 01, ra = 58.4516, dec = -32.9059, m = 19.8203
Retrieving visit table, choosing one visit at random.
Starting source injection.


...

Starting DIA.


Starting source detection.


/opt/lsst/software/stack/conda/envs/lsst-scipipe-10.0.0/lib/python3.12/site-packages/uncertainties/core.py:1024: UserWarning: Using UFloat objects with std_dev==0 may give unexpected results.
  warn("Using UFloat objects with std_dev==0 may give unexpected results.")
/opt/lsst/software/stack/conda/envs/lsst-scipipe-10.0.0/lib/python3.12/site-packages/uncertainties/core.py:1024: UserWarning: Using UFloat objects with std_dev==0 may give unexpected results.
  warn("Using UFloat objects with std_dev==0 may give unexpected results.")


/tmp/ipykernel_33451/3527383882.py:21: RuntimeWarning: invalid value encountered in log10
  -2.5 * np.log10(1 + delta_f / f)
/tmp/ipykernel_33451/3527383882.py:25: RuntimeWarning: invalid value encountered in log10
  -2.5 * np.log10(1 - delta_f / f)


ENDS SIMULATION.


